## 2(3)


In [4]:
import numpy as np
import itertools
from itertools import product
import gurobipy as gp
from gurobipy import GRB

In [6]:
!pip install pulp
!pip install pyomo
import numpy as np


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
demand = {
    1: 20,
    2: 30,
    3: 40,
    4: 35,
    5: 15,
    6: 19,
    7: 25,
    8: 20,
    9: 18,
    10: 12,
    11: 30,
    12: 40
}

capacity = {
    1: 120,
    2: 90,
    3: 100
}
total_capacity = sum(capacity.values())

num_dcs = len(demand)

demand_values = np.array(list(demand.values()))

multipliers = np.array([0.7, 1.0, 1.3])

probabilities = [0.3, 0.4, 0.3]

def s_exceed_prob(num_simulations=1000000):

    randomChoice = np.random.choice(multipliers, size=(num_simulations, num_dcs), p=probabilities)

    simulated_demands = randomChoice * demand_values

    total_demands = simulated_demands.sum(axis=1)

    prob_exceed = np.mean(total_demands > total_capacity)
    return prob_exceed

probability_exceed = s_exceed_prob(num_simulations=1000000)
print(f"Probability of demand exceeding total capacity ({total_capacity}): {probability_exceed:.4f}")


Probability of demand exceeding total capacity (310): 0.3914


## 2(4)

In [9]:
factories = [1, 2, 3]
destinations = list(range(1, 13))
n_scen = 10000
all_scen = list(itertools.product(['low', 'medium', 'high'], repeat=12))
sample_indices = np.random.choice(range(len(all_scen)), size=n_scen, replace=False)
scen_data = []
for idx in sample_indices:
    scn = {}
    for j in destinations:
        r = np.random.rand()
        med = {1: 200000, 2: 300000, 3: 400000, 4: 350000, 5: 150000, 6: 190000, 7: 250000, 8: 200000, 9: 180000, 10: 120000, 11: 300000, 12: 400000}[j]
        if r < 0.3:
            scn[j] = int(0.7 * med)
        elif r < 0.7:
            scn[j] = med
        else:
            scn[j] = int(1.3 * med)
    scen_data.append(scn)
scen_prob = 1.0 / n_scen

cap = {1: 1200000, 2: 900000, 3: 1000000}
cost_prod = {1: 8, 2: 7, 3: 9}
selling_price = 20
salvage_val = 5
penalty_val = 7
ship_cost = {
    (1, 1): 2, (1, 2): 3, (1, 3): 2, (1, 4): 3, (1, 5): 5, (1, 6): 4.4, (1, 7): 3.2, (1, 8): 3, (1, 9): 1.5, (1, 10): 3, (1, 11): 6.0, (1, 12): 1.0,
    (2, 1): 1.9, (2, 2): 3, (2, 3): 2.4, (2, 4): 2.4, (2, 5): 2, (2, 6): 4.1, (2, 7): 2.8, (2, 8): 3, (2, 9): 1.7, (2, 10): 2, (2, 11): 1.0, (2, 12): 4.5,
    (3, 1): 3.1, (3, 2): 1.8, (3, 3): 2.8, (3, 4): 3.4, (3, 5): 2, (3, 6): 2.9, (3, 7): 2.2, (3, 8): 3.2, (3, 9): 2, (3, 10): 2.8, (3, 11): 4.5, (3, 12): 6.0
}

mdl = gp.Model("StochOpt")
prod_vars = mdl.addVars(factories, vtype=GRB.CONTINUOUS, name="prod")
ship_vars = mdl.addVars(n_scen, factories, destinations, vtype=GRB.CONTINUOUS, name="ship")
unsold_vars = mdl.addVars(n_scen, factories, vtype=GRB.CONTINUOUS, name="unsold")
lost_vars = mdl.addVars(n_scen, destinations, vtype=GRB.CONTINUOUS, name="lost")

mdl.setObjective(
    gp.quicksum(
        scen_prob * (
            gp.quicksum(selling_price * ship_vars[s, i, j] for i in factories for j in destinations)
            - gp.quicksum(cost_prod[i] * prod_vars[i] for i in factories)
            - gp.quicksum(ship_cost[i, j] * ship_vars[s, i, j] for i in factories for j in destinations)
            - gp.quicksum(salvage_val * unsold_vars[s, i] for i in factories)
            - gp.quicksum(penalty_val * lost_vars[s, j] for j in destinations)
        ) for s in range(n_scen)
    ),
    GRB.MAXIMIZE
)

for i in factories:
    mdl.addConstr(prod_vars[i] <= cap[i])
for s in range(n_scen):
    for i in factories:
        mdl.addConstr(gp.quicksum(ship_vars[s, i, j] for j in destinations) + unsold_vars[s, i] <= prod_vars[i])
for s in range(n_scen):
    for j in destinations:
        mdl.addConstr(gp.quicksum(ship_vars[s, i, j] for i in factories) + lost_vars[s, j] == scen_data[s][j])
mdl.optimize()
if mdl.status == GRB.OPTIMAL:
    print(f"\nOptimal Expected Profit: {mdl.objVal}")
    print("\nOptimal Production Quantities:")
    for i in factories:
        print(f"Factory {i}: {prod_vars[i].x} units")
    print("\nSample Shipment Plan for Scenario 0:")
    for i in factories:
        for j in destinations:
            if ship_vars[0, i, j].x > 0:
                print(f"Ship {ship_vars[0, i, j].x} units from Factory {i} to Destination {j}")
    print("\nSample Unsold for Scenario 0:")
    for i in factories:
        if unsold_vars[0, i].x > 0:
            print(f"Factory {i} unsold: {unsold_vars[0, i].x} units")
    print("\nSample Lost Sales for Scenario 0:")
    for j in destinations:
        if lost_vars[0, j].x > 0:
            print(f"Destination {j} lost sales: {lost_vars[0, j].x} units")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i3-5005U CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 150003 rows, 510003 columns and 900003 nonzeros
Model fingerprint: 0xa137d5a5
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-04, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+04, 1e+06]
Presolve removed 3 rows and 30000 columns
Presolve time: 1.87s
Presolved: 150000 rows, 480003 columns, 870000 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.10s

Barrier statistics:
 Dense cols : 3
 AA' NZ     : 3.900e+05
 Factor NZ  : 1.650e+06 (roughly 270 MB of memory)
 Factor Ops : 2.095e+07 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual    

## 2(5)

Comparison of Production Quantities at the Three Plants:

(values are in 10000s)

| Plant | Deterministic LP | Stochastic LP (SAA) |
|-------|------------------|---------------------|
| 1     | 120.00           | 120.00              |
| 2     | 90.00            | 90.00               |
| 3     | 94.00            | 100.00              |
